# Heat Map

### fake_heatmap.py

In order to create the heatmap, we need the following packages:

In [1]:
import json
from collections import Counter
import pandas as pd
import altair as alt
import datetime
from dateutil.parser import parse

To read the JSON file that has all the tweets, it is necessary to do:

In [2]:
data = []
with open('dataset/fakecovid_result_translated_full.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))


We're interested in the "created_at" field.

In [3]:
lista = []
index = 0

for element in data: 
    token=data[index]['created_at']
    d = parse(token)
    d = d.strftime('%Y/%m/%d')
    lista.append(d)
    index=index+1

After converting the related date into a string, the number of occurrences in the created 'lista' has been counted.
Then the DataFrame, that is a 2-dimensional labeled data structure, was generated from the array 'lista'.

In [4]:
count=Counter(lista)

start = datetime.datetime.strptime("2020/01/01", "%Y/%m/%d")
end = datetime.datetime.strptime("2020/10/01", "%Y/%m/%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    if date.strftime("%Y/%m/%d") not in count:
        count[date.strftime("%Y/%m/%d")] = 0

df = pd.DataFrame.from_dict(count, orient='index').reset_index()
df = df.rename(columns={'index':'data', 0:'tweet_count'})

The heatmap is created and visualized through the 'altair_viewer':

In [5]:
chart = alt.Chart(
    df,
    title="Tweet count hetmap"
).mark_rect().encode(
    x='date(data):O',
    y='month(data):O',
    color=alt.Color('tweet_count:Q', scale=alt.Scale(scheme="turbo")),
    tooltip=[
        alt.Tooltip('monthdate(data):T', title='Date'),
        alt.Tooltip('tweet_count:Q', title='Tweet Count')
    ]
).properties(width=800, height=300)

# Configure text
text = chart.mark_text(baseline='middle').encode(
    text='tweet_count:Q',
    color=alt.condition(
        alt.datum.num_cars > 100,
        alt.value('black'),
        alt.value('white')
    )
)

chart + text

alt.LayerChart(...)